In [1]:
from nptdms import TdmsFile
import os, shutil, re, csv
import pandas as pd
import numpy as np
import openpyxl

In [2]:
def get_color_row_idx(file):
    workbook = openpyxl.load_workbook(file)
    worksheet = workbook['Individual']
    start_col_idx = 0
    total_col = worksheet.max_column # 48columns need to be extracted
    color_map = openpyxl.styles.colors.COLOR_INDEX
    
    for cell in worksheet['A']:
        if cell.value == 'Time(min)':
            start_row_idx = cell.row
            print("Time(min) start:",start_row_idx) # group1.xlsm shoud be : 50
    for column1 in worksheet.iter_cols(min_col=start_col_idx, max_col = start_col_idx+1, min_row =start_row_idx+1 ):
        for cell in column1:
            color_index = cell.fill.start_color.index
            rgb = color_map[int(color_index)]
            if rgb == '00FF0000':
                end_time_row_idx = cell.row
                print("Index of 1st red BG color:",end_time_row_idx)
                break
    workbook.close()
    return start_row_idx ,end_time_row_idx

In [3]:
fullpath = os.getcwd()
fileall = os.listdir(fullpath)
if 'groupfile.xlsx' in fileall:
    os.remove('groupfile.xlsx')
    print('remove previous groupfile.xlsx')
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]

excelfile = []
wt_data = pd.DataFrame()
a5_data = pd.DataFrame()
p_bout= pd.DataFrame()
group_hour = pd.DataFrame()
group_avg = pd.DataFrame()
individual_nored_data = pd.DataFrame()

for File in os.listdir(fullpath):
        if File.endswith('.xlsm'):
            #print(File)
            excelfile.append(File)
            
for file in excelfile:       
    print("data extracting from : ",file)
    
    # openpyxl read colored cell
    print('Individual data processing ---')
    start_row_idx ,end_row_idx = get_color_row_idx(file)
    individual = pd.read_excel(fullpath+'/'+file, sheet_name = 'Individual')
    individual_nored_data = pd.concat([individual_nored_data,individual.iloc[start_row_idx-2:end_row_idx-2,:]],axis = 1)
    
    # 6 Cycles concating---
    print('6 Cycles data processing---')
    six_Cycles = pd.read_excel(fullpath+'/'+file, sheet_name = '6 Cycles')
    wt_data = pd.concat([wt_data,six_Cycles.loc[:,'WT':'Cycle-5']],axis = 1)
    # extract rows unitl the first blank appears
    wt_data = wt_data.iloc[:wt_data.isnull().any(axis=1).idxmax(),:]
    a5_data = pd.concat([a5_data,six_Cycles.loc[:,'A5':'Cycle-5.1']],axis = 1)
    a5_data = a5_data.iloc[:a5_data.isnull().any(axis=1).idxmax(),:]
    
    # sleep bouts concating ---
    sleepbouts = pd.read_excel(fullpath+'/'+file, sheet_name = 'Sleep Bouts')
    print('Sleep Bouts data processing---')
    id_data1 = sleepbouts[sleepbouts["Unnamed: 0"]== "Data-1"].index.tolist()[0]
    sleepbouts = sleepbouts.iloc[id_data1+1:,:]
    id_data1_end = sleepbouts[sleepbouts["Unnamed: 0"]== "Data-1"].index.tolist()[0]
    #select data with non-null columns and rows
    sleepbouts = sleepbouts.iloc[:id_data1_end-id_data1-1,:]
    sleepbouts = sleepbouts.dropna(axis=1, how = 'all')
    sleepbouts = sleepbouts.dropna(axis=0, how = 'all')
    #print(sleepbouts.shape)    
    p_bout = pd.concat([p_bout,sleepbouts],axis = 1)
    
    # group count data concating ---
    print('groupcount data processing---')
    groupcount = pd.read_excel(fullpath+'/'+file, sheet_name ='GroupCount Data')
    hour_hour = list(groupcount).index("Hour-1 After Correction by Hour")
    hour_avg = list(groupcount).index('Hour-1 After Correction by Average')
    group_hour = pd.concat([group_hour,groupcount.iloc[:,hour_hour:hour_hour+3]],axis = 1)
    group_avg = pd.concat([group_avg,groupcount.iloc[:,hour_avg:hour_avg+3]],axis = 1)

six_cycles_total = pd.concat([wt_data,a5_data],axis = 1)

group_hour.reset_index(drop = True,inplace = True)
group_avg.reset_index(drop = True,inplace = True)
groupc_total = pd.concat([group_hour,group_avg],axis = 1)
print('Grouping DONE!')

with pd.ExcelWriter(fullpath +'/groupfile.xlsx', engine = 'xlsxwriter') as writer:
    
    individual_nored_data.to_excel(writer ,sheet_name = 'Individual',index = False ,header = False)
    six_cycles_total.to_excel(writer ,sheet_name = '6 Cycles',index = False ,header = True)
    p_bout.to_excel(writer ,sheet_name = 'Sleep Bouts',index = False ,header = False)
    groupc_total.to_excel(writer ,sheet_name = 'Group Count Data',index = False ,header = True)
    
    
    worksheet = writer.sheets['6 Cycles']
    worksheet.set_column("A:Z", 10)
    worksheet = writer.sheets['Sleep Bouts']
    worksheet.set_column("A:Z", 15)
    worksheet = writer.sheets['Group Count Data']
    worksheet.set_column("A:Z", 15)
    
writer.save()
writer.close()

remove previous groupfile.xlsx
data extracting from :  TrackingData_Template_v5.7b_2023_group1_Alctreat_0.05%.xlsm
Individual data processing ---
Time(min) start: 50
Index of 1st red BG color: 157
6 Cycles data processing---
Sleep Bouts data processing---
groupcount data processing---
data extracting from :  TrackingData_Template_v5.7b_2023_group2_Alctreat_0.05%.xlsm
Individual data processing ---


H:\Anaconda3_2021\anaconda\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Time(min) start: 50
Index of 1st red BG color: 198
6 Cycles data processing---
Sleep Bouts data processing---
groupcount data processing---
Grouping DONE!


C:\Users\12\AppData\Local\Temp\ipykernel_15848\4162015670.py:85: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
H:\Anaconda3_2021\anaconda\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
